# Biomedical Databases: Design, Implementation and Optimisation

### WS- 2017/2018

## Team 3 lab report

**By**  
Mohammed Abdelgadir Hassan   
Nour Al-hanafi  
Colin Jan Birkenbihl  
Dejan Djukic   
Faridullah Khan  
Gergö Szita   
Lanying Wei  

# 1. Overview
In course of the Biomedical Databases Lab Course project we planned a framework of three communicating databases dedicated to protein - disease, disease - phenotype and protein - pathway interactions. Such a framework could for example be used to annotate pathway data with disease information or to compare diseases based on associated phenotypes or involved proteins (figure. 1).  
As team 3, our part of project comprised mapping proteins to diseases. For this, we aimed at creating a database system that can be easily queried in order for the user to find disease-protein associations (figure. 2). The user can provide either a protein id as an input and obtain all associated diseases as an output or vice versa. Furthermore, the user can query many diseases simultaneously and obtain all proteins linked to theses diseases. 
This may help find novel molecular targets for potential new therapeutics and provide new windows on disease treatments and metabolic process control. 

<img src="db_Interfaces.png">

                                  Figure. 1 Interface between databases

<img src="figure.2.jpg">

                                  Figure. 2 Teams contributions to the databases framework

# 2. Materials & Methods
## 2. 1. Materials
We investigated many biological databases in which data may have included information about both proteins and diseases. We found that UNIPROT had the required information. Raw data existed as a text file containing proteins-ids, proteins names and disease ids (OMIM). 

## 2. 2. Methods
We used python for downloading and parsing the data, and building the database with the help of SQLAlchemy. We mapped OMIM to uniprotIDs and vice versa.   

### 2. 2. 1. Retrieve the raw data
The text file which includes the data is available at : http://www.uniprot.org/docs/mimtosp.txt .
It was downloaded and parsed automatically by a python script. Since the number of relations in each row was not consistent, built-in Pandas functions were not used to parse the document linked above; a custom parser had to be made which delivers the relations as a list of lists to the method which populates the database.  

### 2. 2. 2. Build the data model
The dataset has many to many relationship and to accommodate this, an Association table is set up to link between the Protein and Disease tables. The association table allows for pre-defining the join conditions within the database so that cross-table queries are easily accomplished and Table objects readily obtainable, as demonstrated by two versions of our queries.   
A custom SQLalchemy-based function was created to quickly (5-10 seconds of execution time) populate the database with disease - protein association data, which is also downloaded and parsed automatically by module “mapping_parser_v2” included in our package. We expect the URL of our data to be stable and we have recorded it in the constants file so it could be easily changed when or if the original URL gets broken.   
A schematic of our final disease model is shown below in figure 3 and it is implemented in the “datamodel” module of our package. which holds all three tables as classes which are then imported into our database manager which utilizes our data model implementation for creating, populating and querying the database.  
Even though that the data model is simple, it is easily expandable if need be and more importantly, it fills a critical gap of relating the proteins to diseases whereas most of the available tools (i.e. DisGeNET) holds the associations between genes and diseases.  
  

Figure. 3: Data model schema; Team 3

### 2. 2. 3. Create the database manager
All of the database-related operations are handled from within the “db_manager” module of our package. It is built based on SQLalchemy session-aware design and according to best practices shown by the SQLalchemy documentation.  

### 2. 2. 4. Create the database interactor
The Interactor is a class that combines the three databases of team 1, 2 and 3. It supports SQLalchemy query functions, which query across the different databases.   
An example for these queries would be the annotate_pathway function:  
Using a WikiPathway identifier in a query the genes interacting in the pathway will be retrieved. These genes will then be connected to the Uniprot identifiers of the associated proteins. Using the Uniprot identifiers the database of team 1 will be queried for the disease names associated with these proteins. Thus a link between WikiPathways and diseases is established.  

### 2. 2. 5. Packaging 
We built a project package which could be imported and worked with after installed to make the software useful and user friendly. Such all the important functions can be directly used and the user can leverage from the provided documentation and examples.  

# 3. Results 
## 3. 1. The processed data
The raw (relatively unstructured) text file from Uniprot was successfully processed with through our mapper_parser module and a list of list was returned for further usage in populating the database. After benchmarking the final database performance, we noticed that not all OMIM ID’s in the file actually map to diseases within OMIM and for that, a workaround for filtering out the OMIM ID’s which refer to irrelevant entities was created, based on another .tsv file holding the descriptions of all OMIM ID’s.  

## 3. 2. The database
The final 3-table data model was implemented in SQLalchemy and tested through populating and querying the database on numerous examples. Using SQLalchemy makes it possible to set up the database for different SQL services (MySQL, PostGres
Uniprot accession numbers can successfully be mapped to associated diseases (in particular, their OMIM ID’s) very quickly, in a many-to-many relationship. Please see figure 3 for a detailed schematic of the final data model.  

# 3. 3. The manager
The manager is a python class, which takes care of all the database related functionalities.  
Our database manager is able to create and fill the database. It will create three tables (proteins, diseases and association between them). The populate_db function of the manager will take care of download the raw uniprot data, parsing the data and populating the database with it.   
Functions to query the database using either uniprot id(s) or OMIM id(s) to retrieve data are also implemented in the manager.  

## 3. 3. The interactor
With the interactor class we provided an object, that enables the user to perform more complicated queries across the three databases. It can be used to answer scientific questions, that could not have been answered by looking only into one data resource. Provided functionalities include for example the annotation of WikiPathway pathways with disease names.  

# 3. 4. Annotating pathways with associated diseases
WikiPathway pathways are not linked to any disease information so far. Using our packages and the interactor class we provided the means necessary to accomplish this task. When passing WikiPathway identifiers to the annotate_pathways function of the interactor, the interactor will perform queries (described in more detail in 2.2.4) and then outputs the mapping of the pathway identifier to all diseases it is associated with. This can be done repeatedly and ensures that the annotation is up to date.  

# 4. Conclusion
The package allows for investigating protein-disease relationships so the user can gain knowledge about which diseases relate to which proteins. This can be used as a standalone application and finding interesting proteins regards to a disease. Moreover, the package can be used with the two other packages to show the relationship between disease phenotype and protein pathways.  

# 5. Limitations
The main restriction of our software is that it relies on another package within the framework (namely, group 1 which operates on the Monarch/OMIM database) to provide the user with the full names of the OMIM diseases related to a particular protein, and in a sense, vice versa; It is mainly only protein accession ID's that are returned, not the full protein names.  
This is due to the fact that the package was developed in such a way that it resides in a three-package ecosystem where all packages have a role to fill in reaching the final goal. Because of that, our queries are constructed in such a way that they return the most unique pieces of information (ID's) which is the most useful to other databases and not the full names themselves.  
To somewhat ameliorate the problem, a method in the “db_manager” module with the name “find_protein()” is implemented, it receives a Uniprot accession number as input and returns the name of the related protein.   
Another reason for the lack of descriptiveness in our output is due to the fact that the best and most directly usable data we could find for the protein-disease associations hosted on Uniprot is delivered in such a way that we only have access to Uniprot names of the proteins, which, unfortunately, are not completely user-friendly (i.e. A4_HUMAN is Amyloid-beta A4 protein)  

## 5.1 Future improvements
If we would to, for example, start a master thesis based on a similar approach and improve on what we have done here, we would do the following:  
- We would parse the whole Uniprot xml raw data file and extract much more information from it;  
- We would eliminate the reliance to external packages and develop our own either API handles or local database tables   which contain all of the information required for solving all of the scientific questions in mind  
- We would implement additional algorithms for walking the association search space  

# 6. Contributors
    Colin Jan Birkenbihl
    Dejan Djukic 
    Faridullah Khan
    Gergö Szita 
    Lanying Wei
    Mohammed Abdelgadir Hassan 
    Nour Al-hanafi

